# Downloading Data Into Drive

In [1]:
#@title Mount Drive {form-width: "20%"}
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#@title Set Kaggle Env Var {form-width: "20%"}
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Colab Notebooks/KaggleData/"

In [2]:
#@title cd {form-width: "20%"}
%cd /content/gdrive/MyDrive/Colab Notebooks/KaggleData

/content/gdrive/MyDrive/Colab Notebooks/KaggleData


In [ ]:
#@title Downloading and unziping from Kaggle {form-width: "20%"}
!kaggle datasets download -d hugomathien/soccer
!unzip soccer.zip

# Pandas

In [3]:
#@title Load sqlite into Pandas DF {form-width: "20%"}
import pandas as pd
import sqlite3

con = sqlite3.connect('database.sqlite')


#Loads Match Table
query = '''
select *
from Match
where match.home_player_1 is not null and match.home_player_2 is not null and match.home_player_3 is not null and match.home_player_4 is not null and match.home_player_5 is not null and match.home_player_6 is not null and match.home_player_7 is not null and match.home_player_8 is not null and match.home_player_9 is not null and match.home_player_10 is not null and match.home_player_11 is not null
and match.away_player_1 is not null and match.away_player_2 is not null and match.away_player_3 is not null and match.away_player_4 is not null and match.away_player_5 is not null and match.away_player_6 is not null and match.away_player_7 is not null and match.away_player_8 is not null and match.away_player_9 is not null and match.away_player_10 is not null and match.away_player_11 is not null 
order by league_id, season, stage
'''
Match_dataframe = pd.read_sql_query(query, con)


#Loads Player Table
query = '''
select *
from Player
'''
Player_dataframe = pd.read_sql_query(query, con).set_index('player_api_id')


#Loads Team Table
query = '''
select *
from Team
'''
Team_dataframe = pd.read_sql_query(query, con).set_index('team_api_id')

#Loads League Table
query = '''
select *
from League
'''
League_dataframe = pd.read_sql_query(query, con).set_index('id')

#Loads Country Table
query = '''
select *
from Country
'''
Country_dataframe = pd.read_sql_query(query, con).set_index('id')

In [4]:
#@title Create New DF for Customized Data {form-width: "20%"}
from tqdm.notebook import tqdm
data_columns = ['match_id', 'country', 'league', 'season', 'week', 'date', 'home_team', 'away_team', 'home_goal', 'away_goal', 'result', 'home_lineup', 'away_lineup']
datalist = []
corrupted = 0
strip_chars = '123456789., '
for _, match in tqdm(Match_dataframe.iterrows(), total= Match_dataframe.shape[0]):
  rowlist = [match['id']]
  rowlist.append(Country_dataframe['name'].loc[match['country_id']])
  rowlist.append(League_dataframe['name'].loc[match['league_id']])
  rowlist.append(match['season'])
  rowlist.append(match['stage'])
  rowlist.append(match['date'])
  rowlist.append(Team_dataframe['team_long_name'].loc[match['home_team_api_id']].strip(strip_chars))
  rowlist.append(Team_dataframe['team_long_name'].loc[match['away_team_api_id']].strip(strip_chars))
  rowlist.append(match['home_team_goal'])
  rowlist.append(match['away_team_goal'])
  
  result = match['home_team_goal'] - match['away_team_goal']
  result = 'home' if result>0 else 'tie' if result == 0 else 'away'
  rowlist.append(result)

  hline = []
  aline = []
  for i in range(11):
    hline.append(Player_dataframe['player_name'].loc[match[f"home_player_{i+1}"]].strip(strip_chars))
    aline.append(Player_dataframe['player_name'].loc[match[f"away_player_{i+1}"]].strip(strip_chars))
  
  if len(set(hline + aline)) < 22:
    corrupted += 1
    continue

  rowlist.append(' - '.join(hline))
  rowlist.append(' - '.join(aline))
  #print(rowlist)
  datalist.append(rowlist)

Clean_datafram = pd.DataFrame(datalist, columns=data_columns)

  0%|          | 0/21374 [00:00<?, ?it/s]

In [ ]:
corrupted

65

In [5]:
#@title Save to csv file {form-width: "20%"}
Clean_datafram.to_csv('KaggleDataset.csv', index= False)